In [1]:
### packages required 
!pip install fair-esm
!pip install torch
!pip install tensorflow
!pip install sklearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.1 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import sklearn
print(sklearn.__version__)

1.2.2


## Ensemble Experiment 

In [3]:
import pandas as pd
import numpy as np
import re
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesRegressor
from sklearn.model_selection import LeaveOneOut, cross_validate, cross_val_score, StratifiedShuffleSplit, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score
from sklearn.model_selection import KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from sklearn.metrics import confusion_matrix
import math
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier  # Import Decision Tree
from sklearn.model_selection import GridSearchCV, LeaveOneOut  # Model selection tools


import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

### Base Learner

In [4]:
def SVM(X,Y,x,y):
    param_grid = {
        'C': [0.01, 0.1, 1, 10, 100],  # Regularization parameter
        'kernel': ['linear', 'rbf', 'sigmoid'],  # Removed 'precomputed' kernel
    }
    svm=SVC(random_state=42, probability=True)
    svm_cv = GridSearchCV(svm, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    
    # fitting the model for grid search
    svm_cv.fit(X,Y)
    svm=svm_cv.best_estimator_
    
    print('\n Test Results SVM\n')
    y_p=svm.predict(x)
    
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    
    print('Best accuracy and parameters are: ', svm_cv.best_score_, svm_cv.best_params_)
    accuracy = accuracy_score(y, y_p)
    print("Accuracy:", accuracy)
    print('MCC:', MCC)
    
    return svm, MCC

In [5]:
def Tree(X,Y,x,y):
    #create new a tree model
    DT=DecisionTreeClassifier(random_state=42)
    param_grid = {'max_depth':range(2,20),
                'criterion':['gini', 'entropy']}
    tree_cv = GridSearchCV(DT, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    tree_cv.fit(X, Y)
    tree=tree_cv.best_estimator_
    
    print('\n Test Results Decision Tree\n')
    y_p=tree.predict(x)
    
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    print('Best accuracy and parameters are: ', tree_cv.best_score_, tree_cv.best_params_)
    accuracy = accuracy_score(y, y_p)
    print("Accuracy:", accuracy)
    print('MCC:', MCC)
    
    return tree, MCC

In [6]:
def NaiveBayes(X, Y, x, y):
    # Create a Naive Bayes model
    NB = GaussianNB()
    
    # Define hyperparameter grid (only 'var_smoothing' for GaussianNB)
    param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2]}

    # GridSearchCV with Leave-One-Out Cross-Validation
    nb_cv = GridSearchCV(NB, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    nb_cv.fit(X, Y)
    nb=nb_cv.best_estimator_
    
    print('\n Test Results Naive Bayes\n')
    y_p=nb.predict(x)
    
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    print('Best accuracy and parameters are: ', nb_cv.best_score_, nb_cv.best_params_)
    accuracy = accuracy_score(y, y_p)
    print("Accuracy:", accuracy)
    print('MCC:', MCC)
    
    return nb, MCC

In [7]:
def KNN(X,Y,x,y):
    #create new a knn model
    knn = KNeighborsClassifier()
    #create a dictionary of all values we want to test for n_neighbors
    param_grid = {'n_neighbors': np.arange(1, 10),
                'metric': ['minkowski', 'euclidean', 'manhattan']}
    #use gridsearch to test all values for n_neighbors
    knn_cv = GridSearchCV(knn, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    #fit model to data
    knn_cv.fit(X, Y)
    knn = knn_cv.best_estimator_
    
    print('\n Test Results KNN\n')
    y_p=knn.predict(x)
    
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    print ('Best accuracy and parameters are: ', knn_cv.best_score_, knn_cv.best_params_)
    accuracy = accuracy_score(y, y_p)
    print("Accuracy:", accuracy)
    print('MCC:', MCC)
    return knn, MCC

In [8]:
def LRegression(X,Y,x,y):
    #create new a tree model
    LR=LogisticRegression(random_state=42, max_iter=1500)
    param_grid = {'penalty':['l2', None]}
    
    lr_cv = GridSearchCV(LR, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    lr_cv.fit(X, Y)
    lr=lr_cv.best_estimator_
    
    print('\n Test Results LR\n')
    y_p=lr.predict(x)
    
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    print('Best accuracy and parameters are: ', lr_cv.best_score_, lr_cv.best_params_)
    accuracy = accuracy_score(y, y_p)
    print("Accuracy:", accuracy)
    print('MCC:', MCC)
    
    return lr, MCC

In [9]:
def RandomForest(X,Y,x,y):
    # Number of trees in random forest
    max_depth=range(2,20)
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt', 'log2']
    # Maximum number of levels in tree
    criterion = ['gini', 'entropy']
    
    random_grid = {'max_depth': max_depth,
                'criterion': criterion,
                'max_features': max_features}
    
    rf = RandomForestClassifier(random_state=42)
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                                cv=KFold(10), random_state=42, n_jobs = 4)
    rf_random.fit(X, Y)
    rf = rf_random.best_estimator_
    
    print('\n Test Results RF\n')
    y_p=rf.predict(x)
    
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    print ('Best accuracy and parameters are: ', rf_random.best_score_, rf_random.best_params_)
    accuracy = accuracy_score(y, y_p)
    print("Accuracy:", accuracy)
    print('MCC:', MCC)
    
    return rf, MCC

In [10]:
def ADABoost(X,Y,x,y):
    #create new a tree model
    Ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=42), random_state=42)
    param_grid = {'estimator__max_depth' : [1, 2, 3, 4, 5],
                'estimator__criterion' : ['gini', 'entropy'],
                'estimator__splitter' :   ['best', 'random']}
    ada_cv = GridSearchCV(Ada, param_grid, cv=KFold(10), scoring = 'accuracy', n_jobs=4)
    ada_cv.fit(X,Y)
    ada=ada_cv.best_estimator_
    
    print('\n Test Results ADABoost\n')
    y_p=ada.predict(x)
    
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    print('Best accuracy and parameters are: ', ada_cv.best_score_, ada_cv.best_params_)
    accuracy = accuracy_score(y, y_p)
    print("Accuracy:", accuracy)
    print('MCC:', MCC)
    
    return ada, MCC

In [11]:
def MLPClassify(X,Y,x,y):
    #create new a tree model
    MLP=MLPClassifier(random_state=42, max_iter=1000)
    param_grid = {
      'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # Different sizes of hidden layers
      'activation': ['relu', 'tanh', 'logistic'],  # Activation functions
      'solver': ['adam', 'sgd'],  # Optimization algorithms
      'alpha': [0.0001, 0.001, 0.01],  # L2 regularization parameter
      'learning_rate': ['constant', 'adaptive']  # Learning rate schedule
    }
    
    mlp_cv = GridSearchCV(MLP, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    mlp_cv.fit(X, Y)
    mlp=mlp_cv.best_estimator_
    
    print('\n Test Results MLP\n')
    y_p=mlp.predict(x)
    
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    print('Best accuracy and parameters are: ', mlp_cv.best_score_, mlp_cv.best_params_)
    accuracy = accuracy_score(y, y_p)
    print("Accuracy:", accuracy)
    print('MCC:', MCC)
    
    return mlp, MCC

### ESM Embeddings

In [12]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

In [13]:
# training dataset loading
train = pd.read_excel('/kaggle/input/bitter-esm/bitter_train_filtered_80.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = train['sequence']
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information

# employ ESM model for converting and save the converted data in csv format
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('/kaggle/working/bitter_train_esm.csv')

# loading the y dataset for model development 
y_train = train['label']

Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t6_8M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t6_8M_UR50D-contact-regression.pt


In [14]:
# test dataset loading
test = pd.read_excel('/kaggle/input/bitter-esm/bitter_test_filtered_80.xlsx',na_filter = False)
sequence_list = test['sequence'] 
peptide_sequence_list = []
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information

# employ ESM model for converting and save the converted data in csv format
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('/kaggle/working/bitter_test_esm.csv')

# loading the y dataset for model development 
y_test = test['label']

In [15]:
def ESM():
    # assign the dataset 
    train_path = '/kaggle/working/bitter_train_esm.csv'
    train_data = pd.read_csv(train_path, header=0, index_col = 0, delimiter=',')
    
    test_path = '/kaggle/working/bitter_test_esm.csv'
    test_data = pd.read_csv(test_path,header=0, index_col = 0,delimiter=',')
    
    X_train = np.array(train_data)
    X_test = np.array(test_data)
    
    # normalize the X data range
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train) # normalize X to 0-1 range 
    X_test = scaler.transform(X_test)
    return X_train, X_test

In [16]:
X_train, X_test = ESM()

In [17]:
X_train.shape, X_test.shape

((428, 320), (86, 320))

In [18]:
svm, mcc = SVM(X_train, y_train, X_test, y_test)


 Test Results SVM

Best accuracy and parameters are:  0.83421926910299 {'C': 10, 'kernel': 'rbf'}
Accuracy: 0.872093023255814
MCC: 0.7441107248077677


In [19]:
# train models and store MCCs in a dictionary as follow: {model_name: MCC}
models = {
            'ESM_SVM' : SVM(X_train, y_train, X_test, y_test),
            'ESM_DT' : Tree(X_train, y_train, X_test, y_test),
            'ESM_NB' : NaiveBayes(X_train, y_train, X_test, y_test),
            'ESM_KNN': KNN(X_train, y_train, X_test, y_test),
            'ESM_LR': LRegression(X_train, y_train, X_test, y_test),
            'ESM_RF': RandomForest(X_train, y_train, X_test, y_test),
            'ESM_ADA': ADABoost(X_train, y_train, X_test, y_test),
            'ESM_MLP': MLPClassify(X_train, y_train, X_test, y_test)
         }


 Test Results SVM

Best accuracy and parameters are:  0.83421926910299 {'C': 10, 'kernel': 'rbf'}
Accuracy: 0.872093023255814
MCC: 0.7441107248077677

 Test Results Decision Tree

Best accuracy and parameters are:  0.7100221483942415 {'criterion': 'gini', 'max_depth': 6}
Accuracy: 0.8372093023255814
MCC: 0.6746219776463768

 Test Results Naive Bayes

Best accuracy and parameters are:  0.7358250276854928 {'var_smoothing': 0.01}
Accuracy: 0.8372093023255814
MCC: 0.6819317770742027

 Test Results KNN

Best accuracy and parameters are:  0.794296788482835 {'metric': 'manhattan', 'n_neighbors': 7}
Accuracy: 0.8488372093023255
MCC: 0.6997023700521369

 Test Results LR

Best accuracy and parameters are:  0.8059246954595791 {'penalty': 'l2'}
Accuracy: 0.872093023255814
MCC: 0.7441107248077677

 Test Results RF

Best accuracy and parameters are:  0.8085271317829456 {'max_features': 'log2', 'max_depth': 9, 'criterion': 'entropy'}
Accuracy: 0.8837209302325582
MCC: 0.7711514889173213

 Test Result

### AAE

In [20]:
letters = list("ACDEFGHIKLMNPQRSTVWY")

# replace any character from seqeunce which is not in letters to -
def replace(seq):
    return re.sub('[^' + ''.join(letters) + ']', '-', seq)

train['sequence'] = train['sequence'].apply(replace)
test['sequence'] = test['sequence'].apply(replace)

In [21]:
# AAE
def AAE_1(fastas):
    length = float(len(fastas))
    amino_acids = dict.fromkeys(letters, 0)
    encodings = []
    for AA in amino_acids:
        hits = [a.start() for a in list(re.finditer(AA, fastas))]
        p_prev = 0
        p_next = 1
        sum = 0
        while p_next < len(hits):
            distance = (hits[p_next] - hits[p_prev]) / length
            sum += distance * math.log(distance, 2)
            p_prev = p_next
            p_next += 1
        amino_acids[AA] = -sum
        encodings.append(amino_acids[AA])
    return encodings

def AAE(seq):
    encodings = []
    for fastas in seq:
        # Extracting sub-sequences
        fastas_NT5 = "%s" % fastas[:5]
        fastas_CT5 = "%s" % fastas[-5:]
        # Full sequence encoding
        encodings_full = AAE_1(fastas)
        # Sub-sequence encoding
        encodings_CT5 = AAE_1(fastas_CT5)
        encodings_NT5 = AAE_1(fastas_NT5)
        # Combine encodings
        encodings.append(encodings_full + encodings_NT5 + encodings_CT5)
    return encodings

In [22]:
# Compute encodings
train_encodings_result = AAE(train['sequence'].tolist())
test_encodings_result = AAE(test['sequence'].tolist())

X_train = pd.DataFrame(train_encodings_result)
X_test = pd.DataFrame(test_encodings_result)

In [23]:
X_train.shape, X_test.shape

((428, 60), (86, 60))

In [24]:
# train models and store MCCs in a dictionary as follow: {model_name: MCC}
models.update(
    {
        'AAE_SVM' : SVM(X_train, y_train, X_test, y_test),
        'AAE_DT' : Tree(X_train, y_train, X_test, y_test),
        'AAE_NB' : NaiveBayes(X_train, y_train, X_test, y_test),
        'AAE_KNN': KNN(X_train, y_train, X_test, y_test),
        'AAE_LR': LRegression(X_train, y_train, X_test, y_test),
        'AAE_RF': RandomForest(X_train, y_train, X_test, y_test),
        'AAE_ADA': ADABoost(X_train, y_train, X_test, y_test),
        'AAE_MLP': MLPClassify(X_train, y_train, X_test, y_test)
    }
)


 Test Results SVM

Best accuracy and parameters are:  0.6144518272425249 {'C': 1, 'kernel': 'rbf'}
Accuracy: 0.6976744186046512
MCC: 0.44015540360100736

 Test Results Decision Tree

Best accuracy and parameters are:  0.6215393133997785 {'criterion': 'gini', 'max_depth': 16}
Accuracy: 0.6511627906976745
MCC: 0.3697869984767234

 Test Results Naive Bayes

Best accuracy and parameters are:  0.5983388704318936 {'var_smoothing': 1e-05}
Accuracy: 0.686046511627907
MCC: 0.4193309519544422

 Test Results KNN

Best accuracy and parameters are:  0.5957918050941307 {'metric': 'manhattan', 'n_neighbors': 1}
Accuracy: 0.6395348837209303
MCC: 0.3478516235216784

 Test Results LR

Best accuracy and parameters are:  0.6028792912513843 {'penalty': 'l2'}
Accuracy: 0.6744186046511628
MCC: 0.3982469291220737

 Test Results RF

Best accuracy and parameters are:  0.6237541528239203 {'max_features': 'sqrt', 'max_depth': 5, 'criterion': 'gini'}
Accuracy: 0.6627906976744186
MCC: 0.3912213018462842

 Test Res

### AAI

In [25]:
def AAI_1(fastas):
    encodings = []
    fileAAindex1 = open(R"/kaggle/input/bitter-esm/AAindex_1.txt")
    fileAAindex2 = open(R"/kaggle/input/bitter-esm/AAindex_2.txt")
    records1 = fileAAindex1.readlines()[1:]
    records2 = fileAAindex2.readlines()[1:]
    AAindex1 = []
    AAindex2 = []
    for i in records1:
        AAindex1.append(i.rstrip().split()[1:] if i.rstrip() != "" else None)
    for i in records2:
        AAindex2.append(i.rstrip().split()[1:] if i.rstrip() != "" else None)
    index = {}
    for i in range(len(letters)):
        index[letters[i]] = i
    fastas_len = len(fastas)
    for i in range(len(AAindex1)):
        total = 0
        for j in range(fastas_len):
            temp = AAindex1[i][index[fastas[j]]]
            total = total + float(temp)
        encodings.append(total / fastas_len)
    for i in range(len(AAindex2)):
        total = 0
        for j in range(fastas_len):
            temp = AAindex2[i][index[fastas[j]]]
            total = total + float(temp)
        encodings.append(total)
    return encodings


def AAI(seqs):
    encodings = []
    for fastas in seqs:
        fastas_NT5 = "%s" % fastas[:5]
        fastas_CT5 = "%s" % fastas[-5:]

        encodings_full = AAI_1(fastas)
        encodings_CT5 = AAI_1(fastas_CT5)
        encodings_NT5 = AAI_1(fastas_NT5)
        encodings.append(encodings_full + encodings_NT5 + encodings_CT5)
    return encodings

In [26]:
# Compute encodings
train_encodings_result = AAI(train['sequence'].tolist())
test_encodings_result = AAI(test['sequence'].tolist())

X_train = pd.DataFrame(train_encodings_result)
X_test = pd.DataFrame(test_encodings_result)

In [27]:
X_train.shape, X_test.shape

((428, 36), (86, 36))

In [28]:
# train models and store MCCs in a dictionary as follow: {model_name: MCC}
models.update(
    {
        'AAI_SVM' : SVM(X_train, y_train, X_test, y_test),
        'AAI_DT' : Tree(X_train, y_train, X_test, y_test),
        'AAI_NB' : NaiveBayes(X_train, y_train, X_test, y_test),
        'AAI_KNN': KNN(X_train, y_train, X_test, y_test),
        'AAI_LR': LRegression(X_train, y_train, X_test, y_test),
        'AAI_RF': RandomForest(X_train, y_train, X_test, y_test),
        'AAI_ADA': ADABoost(X_train, y_train, X_test, y_test),
        'AAI_MLP': MLPClassify(X_train, y_train, X_test, y_test)
    }
)


 Test Results SVM

Best accuracy and parameters are:  0.7193798449612403 {'C': 1, 'kernel': 'linear'}
Accuracy: 0.7441860465116279
MCC: 0.48917748917748916

 Test Results Decision Tree

Best accuracy and parameters are:  0.66578073089701 {'criterion': 'gini', 'max_depth': 7}
Accuracy: 0.7093023255813954
MCC: 0.4187178946793119

 Test Results Naive Bayes

Best accuracy and parameters are:  0.5934108527131782 {'var_smoothing': 0.01}
Accuracy: 0.5930232558139535
MCC: 0.18473394347360944

 Test Results KNN

Best accuracy and parameters are:  0.6215393133997785 {'metric': 'manhattan', 'n_neighbors': 8}
Accuracy: 0.686046511627907
MCC: 0.37914871197798117

 Test Results LR

Best accuracy and parameters are:  0.6937430786267995 {'penalty': 'l2'}
Accuracy: 0.7093023255813954
MCC: 0.42025467427280044

 Test Results RF

Best accuracy and parameters are:  0.7287375415282392 {'max_features': 'auto', 'max_depth': 17, 'criterion': 'gini'}
Accuracy: 0.7906976744186046
MCC: 0.5813462701261382

 Test 

### BPNC

In [29]:
# BPNC
def BPNC(seqs):
    encodings = []
    for fastas in seqs:
        fastas_NT5 = "%s" % fastas[:2]
        fastas_CT5 = "%s" % fastas[-2:]
        full = fastas_NT5 + fastas_CT5
        encoding = []
        for AA in full:
            for AA1 in letters:
                tag = 1 if AA == AA1 else 0
                encoding.append(tag)
        encodings.append(encoding)
    return encodings

In [30]:
# Compute encodings
train_encodings_result = BPNC(train['sequence'].tolist())
test_encodings_result = BPNC(test['sequence'].tolist())

X_train = pd.DataFrame(train_encodings_result)
X_test = pd.DataFrame(test_encodings_result)

In [31]:
X_train.shape, X_test.shape

((428, 80), (86, 80))

In [32]:
# train models and store MCCs in a dictionary as follow: {model_name: MCC}
models.update(
    {
        'BPNC_SVM' : SVM(X_train, y_train, X_test, y_test),
        'BPNC_DT' : Tree(X_train, y_train, X_test, y_test),
        'BPNC_NB' : NaiveBayes(X_train, y_train, X_test, y_test),
        'BPNC_KNN': KNN(X_train, y_train, X_test, y_test),
        'BPNC_LR': LRegression(X_train, y_train, X_test, y_test),
        'BPNC_RF': RandomForest(X_train, y_train, X_test, y_test),
        'BPNC_ADA': ADABoost(X_train, y_train, X_test, y_test),
        'BPNC_MLP': MLPClassify(X_train, y_train, X_test, y_test)
    }
)


 Test Results SVM

Best accuracy and parameters are:  0.7801218161683278 {'C': 10, 'kernel': 'rbf'}
Accuracy: 0.813953488372093
MCC: 0.6279841238862575

 Test Results Decision Tree

Best accuracy and parameters are:  0.7336655592469545 {'criterion': 'entropy', 'max_depth': 18}
Accuracy: 0.813953488372093
MCC: 0.6279841238862575

 Test Results Naive Bayes

Best accuracy and parameters are:  0.740420819490587 {'var_smoothing': 0.001}
Accuracy: 0.7558139534883721
MCC: 0.5252647794285569

 Test Results KNN

Best accuracy and parameters are:  0.7591915836101883 {'metric': 'minkowski', 'n_neighbors': 9}
Accuracy: 0.7906976744186046
MCC: 0.5846000738768441

 Test Results LR

Best accuracy and parameters are:  0.7406423034330011 {'penalty': None}
Accuracy: 0.7674418604651163
MCC: 0.5359591602564021

 Test Results RF

Best accuracy and parameters are:  0.7522702104097453 {'max_features': 'log2', 'max_depth': 15, 'criterion': 'entropy'}
Accuracy: 0.872093023255814
MCC: 0.7441107248077677

 Test

### CTD

In [33]:
group1 = {
    "hydrophobicity_PRAM900101": "RKEDQN",
    "normwaalsvolume": "GASTPDC",
    "polarity": "LIFWCMVY",
    "polarizability": "GASDT",
    "charge": "KR",
    "secondarystruct": "EALMQKRH",
    "solventaccess": "ALFCGIVW",
}
group2 = {
    "hydrophobicity_PRAM900101": "GASTPHY",
    "normwaalsvolume": "NVEQIL",
    "polarity": "PATGS",
    "polarizability": "CPNVEQIL",
    "charge": "ANCQGHILMFPSTWYV",
    "secondarystruct": "VIYCWFT",
    "solventaccess": "RKQEND",
}
group3 = {
    "hydrophobicity_PRAM900101": "CLVIMFW",
    "normwaalsvolume": "MHKFRYW",
    "polarity": "HQRKNED",
    "polarizability": "KMHFRYW",
    "charge": "DE",
    "secondarystruct": "GNPSD",
    "solventaccess": "MSPTHY",
}
groups = [group1, group2, group3]
propertys = (
    "hydrophobicity_PRAM900101",
    "normwaalsvolume",
    "polarity",
    "polarizability",
    "charge",
    "secondarystruct",
    "solventaccess",
)


def Count_C(sequence1, sequence2):
    sum = 0
    for aa in sequence1:
        sum = sum + sequence2.count(aa)
    return sum


def Count_D(aaSet, sequence):
    number = 0
    for aa in sequence:
        if aa in aaSet:
            number = number + 1
    cutoffNums = [
        1,
        math.floor(0.25 * number),
        math.floor(0.50 * number),
        math.floor(0.75 * number),
        number,
    ]
    cutoffNums = [i if i >= 1 else 1 for i in cutoffNums]
    code = []
    for cutoff in cutoffNums:
        myCount = 0
        for i in range(len(sequence)):
            if sequence[i] in aaSet:
                myCount += 1
                if myCount == cutoff:
                    code.append((i + 1) / len(sequence))
                    break
        if myCount == 0:
            code.append(0)
    return code


def CTD(seqs):
    encodings = []
    for seq in seqs:
        code = []
        code2 = []
        CTDD1 = []
        CTDD2 = []
        CTDD3 = []
        aaPair = [seq[j : j + 2] for j in range(len(seq) - 1)]
        for p in propertys:
            c1 = Count_C(group1[p], seq) / len(seq)
            c2 = Count_C(group2[p], seq) / len(seq)
            c3 = 1 - c1 - c2
            code = code + [c1, c2, c3]

            c1221, c1331, c2332 = 0, 0, 0
            for pair in aaPair:
                if (pair[0] in group1[p] and pair[1] in group2[p]) or (
                    pair[0] in group2[p] and pair[1] in group1[p]
                ):
                    c1221 = c1221 + 1
                    continue
                if (pair[0] in group1[p] and pair[1] in group3[p]) or (
                    pair[0] in group3[p] and pair[1] in group1[p]
                ):
                    c1331 = c1331 + 1
                    continue
                if (pair[0] in group2[p] and pair[1] in group3[p]) or (
                    pair[0] in group3[p] and pair[1] in group2[p]
                ):
                    c2332 = c2332 + 1
            code2 = code2 + [
                c1221 / len(aaPair),
                c1331 / len(aaPair),
                c2332 / len(aaPair),
            ]
            CTDD1 = CTDD1 + [
                value / float(len(seq)) for value in Count_D(group1[p], seq)
            ]
            CTDD2 = CTDD2 + [
                value / float(len(seq)) for value in Count_D(group2[p], seq)
            ]
            CTDD3 = CTDD3 + [
                value / float(len(seq)) for value in Count_D(group3[p], seq)
            ]
        encodings.append(code + code2 + CTDD1 + CTDD2 + CTDD3)
    return encodings

In [34]:
# Compute encodings
train_encodings_result = CTD(train['sequence'].tolist())
test_encodings_result = CTD(test['sequence'].tolist())

X_train = pd.DataFrame(train_encodings_result)
X_test = pd.DataFrame(test_encodings_result)

y_train = train['label']
y_test = test['label']

In [35]:
X_train.shape, X_test.shape

((428, 147), (86, 147))

In [36]:
svm, mcc = SVM(X_train, y_train, X_test, y_test)


 Test Results SVM

Best accuracy and parameters are:  0.7919712070874861 {'C': 1, 'kernel': 'rbf'}
Accuracy: 0.872093023255814
MCC: 0.7441107248077677


In [37]:
# train models and store MCCs in a dictionary as follow: {model_name: MCC}
models.update(
    {
        'CTD_SVM' : SVM(X_train, y_train, X_test, y_test),
        'CTD_DT' : Tree(X_train, y_train, X_test, y_test),
        'CTD_NB' : NaiveBayes(X_train, y_train, X_test, y_test),
        'CTD_KNN': KNN(X_train, y_train, X_test, y_test),
        'CTD_LR': LRegression(X_train, y_train, X_test, y_test),
        'CTD_RF': RandomForest(X_train, y_train, X_test, y_test),
        'CTD_ADA': ADABoost(X_train, y_train, X_test, y_test),
        'CTD_MLP': MLPClassify(X_train, y_train, X_test, y_test)
    }
)


 Test Results SVM

Best accuracy and parameters are:  0.7919712070874861 {'C': 1, 'kernel': 'rbf'}
Accuracy: 0.872093023255814
MCC: 0.7441107248077677

 Test Results Decision Tree

Best accuracy and parameters are:  0.7266888150609081 {'criterion': 'gini', 'max_depth': 12}
Accuracy: 0.7674418604651163
MCC: 0.5414058996899022

 Test Results Naive Bayes

Best accuracy and parameters are:  0.6168881506090809 {'var_smoothing': 1e-09}
Accuracy: 0.6511627906976745
MCC: 0.3042780159121981

 Test Results KNN

Best accuracy and parameters are:  0.7477297895902547 {'metric': 'minkowski', 'n_neighbors': 3}
Accuracy: 0.8023255813953488
MCC: 0.6065531381256913

 Test Results LR

Best accuracy and parameters are:  0.7638981173864895 {'penalty': 'l2'}
Accuracy: 0.8255813953488372
MCC: 0.6513389472789296

 Test Results RF

Best accuracy and parameters are:  0.7898117386489479 {'max_features': 'log2', 'max_depth': 15, 'criterion': 'entropy'}
Accuracy: 0.8488372093023255
MCC: 0.6975361088445451

 Test 

### DPC

In [38]:
# DPC
diPeptides = [aa1 + aa2 for aa1 in letters for aa2 in letters]


def DPC(seqs):
    encodings = []
    for seq in seqs:
        AADict = {}
        for aa in range(len(letters)):
            AADict[letters[aa]] = aa

        tmpCode = [0] * 400
        for j in range(len(seq) - 2 + 1):
            tmpCode[AADict[seq[j]] * 20 + AADict[seq[j + 1]]] = (
                tmpCode[AADict[seq[j]] * 20 + AADict[seq[j + 1]]] + 1
            )
        if sum(tmpCode) != 0:
            tmpDPC = [i / sum(tmpCode) for i in tmpCode]
        encodings.append(tmpDPC)
    return encodings

In [39]:
# Compute encodings
train_encodings_result = DPC(train['sequence'].tolist())
test_encodings_result = DPC(test['sequence'].tolist())

X_train = pd.DataFrame(train_encodings_result)
X_test = pd.DataFrame(test_encodings_result)

y_train = train['label']
y_test = test['label']

In [40]:
X_train.shape, X_test.shape

((428, 400), (86, 400))

In [41]:
# train models and store MCCs in a dictionary as follow: {model_name: MCC}
models.update(
    {
        'DPC_SVM' : SVM(X_train, y_train, X_test, y_test),
        'DPC_DT' : Tree(X_train, y_train, X_test, y_test),
        'DPC_NB' : NaiveBayes(X_train, y_train, X_test, y_test),
        'DPC_KNN': KNN(X_train, y_train, X_test, y_test),
        'DPC_LR': LRegression(X_train, y_train, X_test, y_test),
        'DPC_RF': RandomForest(X_train, y_train, X_test, y_test),
        'DPC_ADA': ADABoost(X_train, y_train, X_test, y_test),
        'DPC_MLP': MLPClassify(X_train, y_train, X_test, y_test)
    }
)


 Test Results SVM

Best accuracy and parameters are:  0.7638981173864895 {'C': 1, 'kernel': 'rbf'}
Accuracy: 0.813953488372093
MCC: 0.6277056277056277

 Test Results Decision Tree

Best accuracy and parameters are:  0.6143964562569214 {'criterion': 'gini', 'max_depth': 3}
Accuracy: 0.6046511627906976
MCC: 0.27790254988003843

 Test Results Naive Bayes

Best accuracy and parameters are:  0.7218715393133996 {'var_smoothing': 0.01}
Accuracy: 0.7674418604651163
MCC: 0.5357142857142857

 Test Results KNN

Best accuracy and parameters are:  0.6984496124031007 {'metric': 'manhattan', 'n_neighbors': 7}
Accuracy: 0.7209302325581395
MCC: 0.45629995342467633

 Test Results LR

Best accuracy and parameters are:  0.7241971207087485 {'penalty': 'l2'}
Accuracy: 0.7441860465116279
MCC: 0.5125886314693656

 Test Results RF

Best accuracy and parameters are:  0.6937430786267995 {'max_features': 'log2', 'max_depth': 15, 'criterion': 'entropy'}
Accuracy: 0.8255813953488372
MCC: 0.6531277540889141

 Test 

### GTPC

In [42]:
# GTPC
group = {
    "alphatic": "GAVLMI",
    "aromatic": "FYW",
    "postivecharge": "KRH",
    "negativecharge": "DE",
    "uncharge": "STCPNQ",
}
groupKey = group.keys()
tripeptide = [
    g1 + "." + g2 + "." + g3 for g1 in groupKey for g2 in groupKey for g3 in groupKey
]


def GTPC(seqs):
    encodings = []
    for seq in seqs:
        index = {}
        myDict = {}
        for key in groupKey:
            for aa in group[key]:
                index[aa] = key
        for t in tripeptide:
            myDict[t] = 0
        sum = 0
        for j in range(len(seq) - 2):
            myDict[
                index[seq[j]] + "." + index[seq[j + 1]] + "." + index[seq[j + 2]]
            ] = (
                myDict[
                    index[seq[j]] + "." + index[seq[j + 1]] + "." + index[seq[j + 2]]
                ]
                + 1
            )
            sum = sum + 1
        code = []
        if sum == 0:
            for t in tripeptide:
                code.append(0)
        else:
            for t in tripeptide:
                code.append(myDict[t] / sum)
        encodings.append(code)
    return encodings

In [43]:
# Compute encodings
train_encodings_result = GTPC(train['sequence'].tolist())
test_encodings_result = GTPC(test['sequence'].tolist())

X_train = pd.DataFrame(train_encodings_result)
X_test = pd.DataFrame(test_encodings_result)

y_train = train['label']
y_test = test['label']

In [44]:
X_train.shape, X_test.shape

((428, 125), (86, 125))

In [45]:
# train models and store MCCs in a dictionary as follow: {model_name: MCC}
models.update(
    {
        'GTPC_SVM' : SVM(X_train, y_train, X_test, y_test),
        'GTPC_DT' : Tree(X_train, y_train, X_test, y_test),
        'GTPC_NB' : NaiveBayes(X_train, y_train, X_test, y_test),
        'GTPC_KNN': KNN(X_train, y_train, X_test, y_test),
        'GTPC_LR': LRegression(X_train, y_train, X_test, y_test),
        'GTPC_RF': RandomForest(X_train, y_train, X_test, y_test),
        'GTPC_ADA': ADABoost(X_train, y_train, X_test, y_test),
        'GTPC_MLP': MLPClassify(X_train, y_train, X_test, y_test)
    }
)


 Test Results SVM

Best accuracy and parameters are:  0.6965116279069768 {'C': 1, 'kernel': 'linear'}
Accuracy: 0.686046511627907
MCC: 0.3832120403378151

 Test Results Decision Tree

Best accuracy and parameters are:  0.65437430786268 {'criterion': 'gini', 'max_depth': 12}
Accuracy: 0.5930232558139535
MCC: 0.22244170101371633

 Test Results Naive Bayes

Best accuracy and parameters are:  0.6684385382059801 {'var_smoothing': 0.01}
Accuracy: 0.686046511627907
MCC: 0.37914871197798117

 Test Results KNN

Best accuracy and parameters are:  0.6636766334440753 {'metric': 'minkowski', 'n_neighbors': 7}
Accuracy: 0.6162790697674418
MCC: 0.24900558852795376

 Test Results LR

Best accuracy and parameters are:  0.6918604651162792 {'penalty': 'l2'}
Accuracy: 0.6627906976744186
MCC: 0.32926108748531563

 Test Results RF

Best accuracy and parameters are:  0.6942414174972313 {'max_features': 'log2', 'max_depth': 15, 'criterion': 'entropy'}
Accuracy: 0.6627906976744186
MCC: 0.3459036908510489

 Te

In [46]:
models

{'ESM_SVM': (SVC(C=10, probability=True, random_state=42), 0.7441107248077677),
 'ESM_DT': (DecisionTreeClassifier(max_depth=6, random_state=42),
  0.6746219776463768),
 'ESM_NB': (GaussianNB(var_smoothing=0.01), 0.6819317770742027),
 'ESM_KNN': (KNeighborsClassifier(metric='manhattan', n_neighbors=7),
  0.6997023700521369),
 'ESM_LR': (LogisticRegression(max_iter=1500, random_state=42),
  0.7441107248077677),
 'ESM_RF': (RandomForestClassifier(criterion='entropy', max_depth=9, max_features='log2',
                         random_state=42),
  0.7711514889173213),
 'ESM_ADA': (AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=5,
                                                      random_state=42),
                     random_state=42),
  0.721259831406496),
 'ESM_MLP': (MLPClassifier(max_iter=1000, random_state=42),
  0.8177893426774406),
 'AAE_SVM': (SVC(C=1, probability=True, random_state=42), 0.44015540360100736),
 'AAE_DT': (DecisionTreeClassifier(max_depth=16, random_

In [47]:
# Sort the models dictionary by MCC value in descending order
top_models = sorted(models.items(), key=lambda item: item[1][1], reverse=True)[:8]

# Convert the result into a dictionary if needed
top_models_dict = dict(top_models)

# Print the top 8 models
print("Top 8 models based on MCC values:")
for model_name, (model, mcc) in top_models:
    print(f"Model: {model_name}, MCC: {mcc}")

Top 8 models based on MCC values:
Model: ESM_MLP, MCC: 0.8177893426774406
Model: ESM_RF, MCC: 0.7711514889173213
Model: ESM_SVM, MCC: 0.7441107248077677
Model: ESM_LR, MCC: 0.7441107248077677
Model: BPNC_RF, MCC: 0.7441107248077677
Model: CTD_SVM, MCC: 0.7441107248077677
Model: ESM_ADA, MCC: 0.721259831406496
Model: ESM_KNN, MCC: 0.6997023700521369


In [ ]:
# # Sort the models dictionary by MCC value in descending order
# top_models = sorted(models.items(), key=lambda item: item[1][1], reverse=True)[:6]

# # Convert the result into a dictionary if needed
# top_models_dict = dict(top_models)

# # Print the top 8 models
# print("Top 8 models based on MCC values:")
# for model_name, (model, mcc) in top_models:
#     print(f"Model: {model_name}, MCC: {mcc}")

## Generate Probability Dataset

In [48]:
# mapping for encoding function names to actual functions
encoding_function_map = {
    "AAE": AAE,
    "AAI": AAI,
    "BPNC": BPNC,
    "CTD": CTD,
    "DPC": DPC,
    "GTPC": GTPC,
    "ESM": ESM
}

def compute_mcc(y,y_p):
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))
    return MCC
    
def generate_probabilities(train, test, encoding_func_name, encoding_func, classifier):
    # Compute encodings
    print("Classifier: ", classifier)
    if encoding_func_name == "ESM":
        print("Using ESM encoding")
        X_train, X_test = encoding_func()
    else:
        train_encodings_result = encoding_func(train['sequence'].tolist())
        test_encodings_result = encoding_func(test['sequence'].tolist())
        
        X_train = pd.DataFrame(train_encodings_result)
        X_test = pd.DataFrame(test_encodings_result)
    
    y_train = train['label']
    y_test = test['label']

    # Check if classifier supports predict_proba or decision_function
    if hasattr(classifier, "predict_proba"):
        print("PREDICT PROB")
        train_proba = classifier.predict_proba(X_train)[:, 1]  # Get probability for positive class
        test_proba = classifier.predict_proba(X_test)[:, 1]
    elif hasattr(classifier, "decision_function"):
        print("Decision")
        train_proba = classifier.decision_function(X_train)  # Use decision function output
        test_proba = classifier.decision_function(X_test)
    else:
        raise ValueError(f"Classifier {classifier.__class__.__name__} does not support probability estimation.")

    y_train_p = classifier.predict(X_train)
    y_test_p = classifier.predict(X_test)
    
    Train_MCC = compute_mcc(y_train, y_train_p)
    Test_MCC = compute_mcc(y_test, y_test_p)

    print(f"MCC values for encoding {encoding_func} and classifier {classifier.__class__.__name__}")
    print("Training set", Train_MCC)
    print("Test set", Test_MCC)
    
    return train_proba, test_proba


In [49]:
# Initialize empty DataFrames outside the loop
train_prob_df = pd.DataFrame()
test_prob_df = pd.DataFrame()

for key in top_models_dict.keys():
    classifier = top_models_dict[key][0]
    encoding_func_name = key.split("_")[0]
    
    # Get the actual encoding function from the map
    encoding_func = encoding_function_map.get(encoding_func_name)
    if encoding_func is None:
        raise ValueError(f"Encoding function {encoding_func_name} not found in the map.")

    train_proba, test_proba = generate_probabilities(train, test, encoding_func_name, encoding_func, classifier)

    # Update train_prob_df
    train_prob_df[key] = train_proba

    # Update test_prob_df
    test_prob_df[key] = test_proba

Classifier:  MLPClassifier(max_iter=1000, random_state=42)
Using ESM encoding
PREDICT PROB
MCC values for encoding <function ESM at 0x7f5e29067e20> and classifier MLPClassifier
Training set 0.7202210730562499
Test set 0.8177893426774406
Classifier:  RandomForestClassifier(criterion='entropy', max_depth=9, max_features='log2',
                       random_state=42)
Using ESM encoding
PREDICT PROB
MCC values for encoding <function ESM at 0x7f5e29067e20> and classifier RandomForestClassifier
Training set 1.0
Test set 0.7711514889173213
Classifier:  SVC(C=10, probability=True, random_state=42)
Using ESM encoding
PREDICT PROB
MCC values for encoding <function ESM at 0x7f5e29067e20> and classifier SVC
Training set 0.9813370384863206
Test set 0.7441107248077677
Classifier:  LogisticRegression(max_iter=1500, random_state=42)
Using ESM encoding
PREDICT PROB
MCC values for encoding <function ESM at 0x7f5e29067e20> and classifier LogisticRegression
Training set 0.7521823261755706
Test set 0.7441

In [50]:
# Join the label column with train_prob_df
train_prob_df = pd.concat([train_prob_df, train['label'].reset_index(drop=True)], axis=1)

test_prob_df = pd.concat([test_prob_df, test['label'].reset_index(drop=True)], axis=1)

In [52]:
train_prob_df.to_csv('/kaggle/working/train_probability_bitter_top_8.csv', index=False)

In [53]:
test_prob_df.to_csv('/kaggle/working/test_probability_bitter_top_8.csv', index=False)

In [54]:
train_prob_df.head()

,ESM_MLP,ESM_RF,ESM_SVM,ESM_LR,BPNC_RF,CTD_SVM,ESM_ADA,ESM_KNN,label
0,0.208869,0.225916,0.120473,0.325637,0.239663,0.201686,3.282021e-07,0.285714,0
1,0.561328,0.916570,0.890509,0.625433,0.794804,0.650187,9.999997e-01,0.714286,1
2,0.966768,0.855556,0.976140,0.966847,0.774086,0.936066,1.000000e+00,0.857143,1
3,0.252922,0.179127,0.120541,0.222454,0.100306,0.540863,4.104141e-06,0.428571,0
4,0.808304,0.841395,0.890420,0.733661,0.734144,0.965677,9.999998e-01,0.714286,1


In [55]:
test_prob_df.head()

,ESM_MLP,ESM_RF,ESM_SVM,ESM_LR,BPNC_RF,CTD_SVM,ESM_ADA,ESM_KNN,label
0,0.889218,0.693474,0.717101,0.891545,0.831900,0.955290,9.999980e-01,0.714286,1
1,0.139444,0.374270,0.153677,0.403338,0.433875,0.122790,4.213090e-07,0.428571,0
2,0.082656,0.162166,0.048089,0.086729,0.219467,0.049946,5.067264e-05,0.000000,0
3,0.027806,0.209480,0.041478,0.046651,0.234724,0.070429,2.934512e-04,0.000000,0
4,0.003090,0.147000,0.025731,0.012326,0.147277,0.107602,3.981436e-09,0.000000,0


## Meta Learner

In [56]:
def LRegression(X,Y,x,y):
    #create new a tree model
    LR=LogisticRegression(random_state=42, max_iter=1500)
    param_grid = {'penalty':['l2', None]}
    
    lr_cv = GridSearchCV(LR, param_grid, scoring='accuracy', cv=KFold(10), n_jobs=4)
    lr_cv.fit(X, Y)
    lr=lr_cv.best_estimator_
    
    print('\n Test Results LR\n')
    y_p=lr.predict(x)
    predicted_protability = lr.predict_proba(x)
    TP, FP, FN, TN = confusion_matrix(y, y_p).ravel()
    
    ACC = (TP + TN) / (TP + TN + FP + FN)
    Sn = TP / (TP + FN)
    Sp = TN / (TN + FP)
    BACC = 0.5 * TP / (TP + FN) + 0.5 * TN / (TN + FP)
    AUC = roc_auc_score(y, predicted_protability[:, 1])
    MCC = (TP * TN - FP * FN) / math.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN))

    return lr, ACC, Sn, Sp, BACC, AUC, MCC

In [104]:
# Assuming you have X_train, y_train, X_test, y_test defined
# Call the LRegression function
lr, ACC, Sn, Sp, BACC, AUC, MCC = LRegression(train_prob_df.drop('label', axis=1), 
                                          train_prob_df['label'], 
                                          test_prob_df.drop('label', axis=1), 
                                          test_prob_df['label'])

# Print the results
print("Meta Learner Results:")
print(f"Accuracy (ACC): {ACC:.4f}")
print(f"Sensitivity (Sn): {Sn:.4f}")
print(f"Specificity (Sp): {Sp:.4f}")
print(f"Balanced Accuracy (BACC): {BACC:.4f}")
print(f"Area Under Curve (AUC): {AUC:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {MCC:.4f}")


 Test Results LR

Meta Learner Results:
Accuracy (ACC): 0.9531
Sensitivity (Sn): 0.9531
Specificity (Sp): 0.9531
Balanced Accuracy (BACC): 0.9531
Area Under Curve (AUC): 0.9836
Matthews Correlation Coefficient (MCC): 0.9062


In [57]:
import json

# Convert the model dictionary to a JSON-serializable format
serializable_model = {key: (str(value[0]), value[1]) for key, value in models.items()}

# Save to a JSON file
with open("model_results.json", "w") as f:
    json.dump(serializable_model, f, indent=4)

print("Model results saved to model_results.json")

Model results saved to model_results.json


In [58]:
# Convert the model dictionary to a JSON-serializable format
serializable_model = {key: (str(value[0]), value[1]) for key, value in top_models_dict.items()}

# Save to a JSON file
with open("top_8_model_results.json", "w") as f:
    json.dump(serializable_model, f, indent=4)

print("Model results saved to model_results.json")

Model results saved to model_results.json


In [59]:
import joblib
import json
import os

# Directory to save models
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# Dictionary to store MCC scores
mcc_scores = {}

for model_name, (model, mcc) in top_models_dict.items():
    # Save model
    model_path = os.path.join(save_dir, f"{model_name}.pkl")
    joblib.dump(model, model_path)
    
    # Store MCC score
    mcc_scores[model_name] = mcc

# Save MCC scores to a JSON file
mcc_path = os.path.join(save_dir, "mcc_scores.json")
with open(mcc_path, "w") as f:
    json.dump(mcc_scores, f, indent=4)

print(f"Models and MCC scores saved in '{save_dir}'")

Models and MCC scores saved in 'saved_models'
